# Notebook description

This notebook takes as inputs two files, called 'counts_ginis.csv' and 'queries_origin_matched.csv' and produces as output a file called 'event_summary.csv' which is used in the actual analysis.

First, the script reads data about the query-events from counts_ginis.csv. Then, it uses data from queries_origin_matched.csv to filter the data so that only a certain number of days before and after the event's time of origin are considered.

Note that there is no 1-to-1 mapping between Futusome events and queries, so a query-event may have multiple origin dates. Due to this, the script checks if two or more origin times that correspond to a query are included within the time window mentioned above. If so, the queries are considered to form a single event. Otherwise, they are treated as different events.

In [ ]:
import csv
import pandas as pd
import datetime
import collections
import sys

csv.field_size_limit(sys.maxsize)

Essentially, we're interested in looking at some number of days before and after an event's time of origin. This part can be used to set up these parameters; the paper used days_before = 0 and days_before = 30. Also set up the final date to be used, which here is 2017-05-17.

In [ ]:
days_before = 0
days_after = 30
interval = datetime.timedelta(days = days_before + days_after)
#interval = datetime.timedelta(days = 30)
final_date = datetime.date(2017, 5, 17)

The event queries contain some incorrect characters, this is used to correct them. Note that the characters here are not the regular 'ö', 'ä', 'Ö' and 'Ä' although they look like them.

In [ ]:
wrong_ae = 'ä'
wrong_oe = 'ö'
wrong_OE = 'Ö'
wrong_AE = '̈A'

# Read data and combine events

Read queries and corresponding data. This script also replaces the faulty characters mentioned above with '*'s.

In [ ]:
def read_numbers(path):

    ndict = collections.defaultdict(dict)

    with open(path, 'r') as f:

        reader = csv.DictReader(f, delimiter = ',')

        for row in reader:

            ## The one row for each query and each value type
            ## Let's split the index and fix broken letters
            #query_count = row['query / count'].replace(wrong_ae, 'ä').replace(wrong_AE, 'Ä').replace(wrong_oe, 'ö').replace(wrong_OE, 'Ö')

            ## This is 'query / type' for 15062017 onwards,
            ## 'query / count' before that
            query_count = row['query / type'].replace(wrong_ae, '*').replace(wrong_AE, '*').replace(wrong_oe, '*').replace(wrong_OE, '*')
            query_count = query_count.split(' / ')
            query = query_count[0]
            value_type = query_count[1]
            row.pop('query / type')
            ndict[query][value_type] = row
            
    return ndict

This function checks if two or more origin times that map to a query are included within the same time window and, if so, combines them. Used as a helper function by select_days().

In [ ]:
def combine_events(orig_ats, event_ids, interval):
    
    i = 0
    j = 1

    clean_origins = set()

    while True:

        ## Here we'll also add an id to each event

        if j >= len(orig_ats):
            if len(orig_ats) == 1:
                clean_origins.add((orig_ats[0], event_ids[0]))
            break

        first = orig_ats[i]
        first_id = event_ids[i]
        second = orig_ats[j]
        second_id = event_ids[j]

        if second - first <= interval:
            clean_origins.add((first, first_id))
            j += 1
        else:
            clean_origins.add((first, first_id))
            clean_origins.add((second, second_id))
            i += 1
            j += 1
            
    return clean_origins

Loops through the data associated with a query. Discards a query if some of the data is missing. Otherwise takes each part of the event data (i.e. information about sources, authors etc. per day) as a dict, appends the events origin time, event id, corresponding query and data type and returns a list containing these dicts.

In [ ]:
def loop_query_data(query_data, orig_at, event_id, _query):

    origs_to_add = []
    
    for k,v in query_data.iteritems():
        
        ## If every type of count (posts count, domains count)
        ## has something other than zero in the first slot, 
        ## the event will be added to the list. If it has a zero,
        ## something's broken and the event will be discarded
        ## and its query printed.

        ## The current data set has three broken events, I believe.

        v = v.copy()

        if v[str(orig_at)] == str(0):
            print 'Error at: ' + _query
            return False, origs_to_add

        v['orig_at'] = orig_at
        v['event_id'] = event_id
        v['query'] = _query
        v['value_type'] = k

        origs_to_add.append(v)
        
    return True, origs_to_add

This function selects data from days falling within the time interval specified above from an event's origin. It also replaces the faulty characters mentioned earlier with '*'s. 

In [ ]:
def select_days(path, numdict):

    events = []

    ## How many days before and after origin at are looked at        

    with open(path, 'r') as f:

        reader = csv.DictReader(f, delimiter = ',')

        for row in reader:
            query = row['query']
            query = row['query'].replace('ö', '*').replace('ä', '*').replace('Ö', '*').replace('Ä', '*')
            
            ## Here, there may be multiple origin dates and event ids
            ## for a given query, so let's separate them
            event_ids = row['id'].split(';')
            orig_ats = row['orig_at'].split(';')
            orig_dates = []

            ## Consider each origin_at date. If the temporal overlap
            ## between two origin_ats related to an event is large enough,
            ## treat it as multiple independent events.

            for i in range(0, len(orig_ats)):
                ## Clean out milliseconds
                orig_at = orig_ats[i].split('.')[0]
                orig_at = datetime.datetime.strptime(orig_at, '%Y-%m-%d %H:%M:%S').date()
                orig_ats[i] = orig_at

            clean_origins = combine_events(orig_ats, event_ids, interval)

            ## Loop through each event 'version'
            version = 0

            for origin in clean_origins:

                ## This part makes sure that only events that
                ## do not have incomplete data associated with them
                ## are considered
                
                ## Consider each event and each id
                orig_at = origin[0]
                event_id = origin[1]

                ## Fetch related data
                query_data = numdict[query]
                ## In case of multiple events correspond to a single query,
                ## append 'version number' to the query name.
                _query = query + '_' + str(version)

                ## Go through the data associated with the query.
                ## If loop_query_data() returns True, there were no
                ## problems with the data so it's added to events.
                success, origs_to_add = loop_query_data(query_data, orig_at, event_id, _query)

                if success:
                    for orig in origs_to_add:
                        events.append(orig)

                version += 1
                
    return events

In [ ]:
e = read_numbers('data/csv/counts_ginis_2017-08-27.csv')
e = select_days('data/csv/queries_orig_matched_2017-08-24.csv', e)

# Turn data into a data frame

In [ ]:
df = pd.DataFrame(e)
df = df.set_index(['query', 'value_type'])
df.drop('all documents', axis = 1, inplace = True)
df.drop('event_id', axis = 1, inplace = True)

In [ ]:
def selected_days(columns):
    orig_at = columns.loc['orig_at']
    dates = columns.index
    dates = dates.drop('orig_at')
    
    selected = []
    
    if orig_at + datetime.timedelta(days = days_after) > final_date:
        return
    
    for date in dates:
        column_date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
        if column_date >= orig_at - datetime.timedelta(days = days_before) \
        and column_date < orig_at + datetime.timedelta(days = days_after):
            if 'counts' in columns.name[1]:
                selected.append(columns.loc[date])
                    
    return pd.Series(selected)

In [ ]:
df = df.iloc[df.index.get_level_values('value_type').str.contains('counts')]

In [ ]:
df = df.apply(selected_days, axis = 1)

# Form event summary file

Find out how many days an event lasted, i.e. how many days it took for post count to drop to zero.

In [ ]:
def get_event_duration(columns):
    
    if (columns == '0').any() == False:
        return 30
    return int((columns == '0').argmax())

Gets the total of dataframe values during the time an event lasted, and the average.

In [ ]:
def total_during_duration(columns):
    
    duration = columns['duration']
    
    active_days = columns[0:duration]
    
    return sum(active_days.astype(float))

def average_during_duration(columns):
    
    duration = columns['duration']
    
    active_days = columns[0:duration]
    
    return sum(active_days.astype(float)) / duration

Manually separate the dataframe.

In [ ]:
posts_df = df.loc[(df.index.get_level_values('value_type') == 'post counts')]
posts_df = posts_df.reset_index().drop('value_type', axis = 1).set_index('query')

authors_df = df.loc[(df.index.get_level_values('value_type') == 'author counts')]
authors_df = authors_df.reset_index().drop('value_type', axis = 1).set_index('query')

domains_df = df.loc[(df.index.get_level_values('value_type') == 'domain counts')]
domains_df = domains_df.reset_index().drop('value_type', axis = 1).set_index('query')

sources_df = df.loc[(df.index.get_level_values('value_type') == 'source counts')]
sources_df = sources_df.reset_index().drop('value_type', axis = 1).set_index('query')

Apply the dataframe functions defined above.

In [ ]:
duration_df = posts_df.apply(get_event_duration, axis = 1)  

In [ ]:
## Rename some columns

posts_df.columns = [str(i) + ' posts' for i in range(0,30)]
authors_df.columns = [str(i) + ' authors' for i in range(0,30)]
domains_df.columns = [str(i) + ' domains' for i in range(0,30)]
sources_df.columns = [str(i) + ' sources' for i in range(0,30)]

## Add duration info to each data frame

posts_df['duration'] = duration_df
authors_df['duration'] = duration_df
domains_df['duration'] = duration_df
sources_df['duration'] = duration_df

In [ ]:
## Compute the total and average values of the variables
## during the time the event was 'active'

posts_df['total posts'] = posts_df.apply(total_during_duration, axis = 1)
posts_df['average posts'] = posts_df.apply(average_during_duration, axis = 1)

authors_df['total authors'] = authors_df.apply(total_during_duration, axis = 1)
authors_df['average authors'] = authors_df.apply(average_during_duration, axis = 1)

domains_df['total domains'] = domains_df.apply(total_during_duration, axis = 1)
domains_df['average domains'] = domains_df.apply(average_during_duration, axis = 1)

sources_df['total sources'] = sources_df.apply(total_during_duration, axis = 1)
sources_df['average sources'] = sources_df.apply(average_during_duration, axis = 1)

Recombine data frame.

In [ ]:
## Combine the data into a single data frame

combine_df = posts_df[['0 posts', '1 posts', '2 posts', 'total posts', 'average posts']]
combine_df = pd.concat([combine_df, authors_df[['0 authors', '1 authors', '2 authors', 'total authors', 'average authors']]], axis = 1)
combine_df = pd.concat([combine_df, domains_df[['0 domains', '1 domains', '2 domains', 'total domains', 'average domains']]], axis = 1)
combine_df = pd.concat([combine_df, sources_df[['0 sources', '1 sources', '2 sources', 'total sources', 'average sources']]], axis = 1)
combine_df = pd.concat([combine_df, posts_df[['duration']]], axis = 1)

Turn query name into event name and event type

Also removes things like 'AND NOT text.exact' as well
as the marker for different 'iterations' of the same event,
e.g. 'ykk*saamu_0' and 'ykk*saamu_1' both become 'ykk*saamu'

Possible event types are hashtag and substantive (i.e. keyword)

In [ ]:
def split_event_name_and_type(columns):
    query = columns['query']
    
    split = query.split(':')
    event_type = split[0].split('.')[1]
    event_name = split[1]
    event_name = event_name.split(' ')[0]
    event_name = event_name.rsplit('_')[0]
    return pd.Series({'event name': event_name, 'event type': event_type})

combine_df = combine_df.reset_index() 
combine_df[['event name', 'event type']] = combine_df.reset_index().apply(split_event_name_and_type, axis = 1)

The next three blocks wrangle the data frame, show it for inspection, and write it in a file.

In [ ]:
combine_df = combine_df.drop('query', axis = 1)
event_names = combine_df['event name']
event_types = combine_df['event type']
combine_df = combine_df.drop(['event name', 'event type'], axis = 1)
combine_df.insert(0, 'event type', event_types)
combine_df.insert(0, 'event name', event_names)
combine_df = combine_df.set_index('event name')

In [ ]:
combine_df

In [ ]:
## And output it. Remember to set the proper file name!

combine_df.to_csv('data/csv/event_summary.csv')